In [6]:
setwd('..')
getwd()


[1] "c:/Users/hso20/OneDrive/Plocha/IES/Asset pricing"

In [1]:
read.csv('data/group_names.csv')

ï..Group.Name
22138943
43512365
93419523
88952997
61505008
96640370
18186923
12870482
20107894
24766912


# JEM092 Asset Pricing - Homework 2
### Summer Semester 2021/2022

### Authors: *Petr Čala, Tereza Čechová, Vilém Krejcar*
___

First things first, we install and load the necessary packages. Then we arbitrarily set a seed for reproducibility.

In [2]:
# Required packages
packages <- c("quantmod", "tseries", "dplyr", "xml2", "httr", "PortfolioAnalytics")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

### Groundwork

Although we were able to obtain the data in the first homework, **we choose to use the provided dataset**, as this allows for easier and **more reliable reproducibility**. First we load the data, subset the necessary stocks, save these as new variables, and then discard the source data in order to alleviate some memory.

In [3]:
# Static variables
group_id <- "61505008"
required_file_names <- c("rand_download")
exp_len <- 20 #Number of stocks to be analyzed

In [4]:
# Assert presence of necessary files
missing = 0
for (i in required_file_names) {
    name <- paste(group_id, "_", i, ".csv", sep = "")
    if (!name %in% list.files()) {
        print(paste("File ", name, " is missing from the folder.", sep = ""))
        missing <- missing + 1
    }
}
if (missing == 0) {
    print('All required files were located successfully.')
}

[1] "All required files were located successfully."


In [5]:
# A function for loading the tickers from a file
getTickers <- function(file_name) {
    tickers <- as.character(read.csv(paste(group_id, "_", file_name, ".csv", sep = ""))[,2]) # Load the tickers
    if (length(tickers) != exp_len) {
        print(paste("There seem to be some missing tickers. There should be ", exp_len, " of them in total.", sep = ""))
    }
    return(tickers)
}
tickers <- getTickers("rand_download") # We of course load the tickers as so

In [22]:
# Load source data into the working directory
source_data <- load('Asset_Pricing_HW_2_data.RData')

# Subset only for the stocks we will need
BV <- book_value_sap500[tickers]
MKT <- MktCap_sap500[tickers]
OHLCV <- OHLCV_sap500[tickers]

# Remove source data from working directory for better memory management
rm(list = source)

### Portfolio analysis